CREATING THE DATASET

### Import and Install modules

In [ ]:
!pip install nltk faker transformers datasets tokenizers seqeval transformers[torch] accelerate -U


## Mount the google drive

In [ ]:
from google.colab import drive
drive.mount('REDACTED')


## Import the formatted_dataset for training

```
`# This is formatted as code`
```



In [ ]:
import json

with open('REDACTED', 'r') as file:
    your_dataset = json.load(file)


## Split the dataset into 3: Train, Validation and test.

(If not already Done)

In [ ]:
import json
import random

# Load your dataset
with open('REDACTED', 'r') as f:
    dataset = json.load(f)

# Shuffle and split the dataset
random.shuffle(dataset)
train_size = int(0.8 * len(dataset))
valid_size = int(0.1 * len(dataset))

train_data = dataset[:train_size]
valid_data = dataset[train_size:train_size+valid_size]
test_data = dataset[train_size+valid_size:]

# Create DatasetDict format
def format_dataset(data):
    formatted_data = {
        "id": [item["id"] for item in data],
        "tokens": [item["tokens"] for item in data],
        "pos_tags": [item["pos_tags"] for item in data],
        "chunk_tags": [item["chunk_tags"] for item in data],
        "ner_tags": [item["ner_tags"] for item in data]
    }
    return formatted_data

dataset_dict = {
    "train": format_dataset(train_data),
    "validation": format_dataset(valid_data),
    "test": format_dataset(test_data)
}

# Write the dataset_dict to a new JSON file
with open('REDACTED', 'w') as f:
    json.dump(dataset_dict, f, indent=4)


In [ ]:
from google.colab import drive
drive.mount('REDACTED')

## Import the necessary modules

In [ ]:
import datasets
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
import json
from datasets import load_dataset



Import the Datasets with correct features.

This script makes a new dataset after called new_dataset. this is the one which will be used

In [ ]:
from datasets import Dataset, DatasetDict, ClassLabel, Features, Sequence, Value

# Load dataset_dict from the JSON file
with open('REDACTED', 'r') as f:
    dataset_dict = json.load(f)

# Assuming you have already loaded dataset_dict from the JSON file:
dataset = DatasetDict({
    "train": Dataset.from_dict(dataset_dict["train"]),
    "validation": Dataset.from_dict(dataset_dict["validation"]),
    "test": Dataset.from_dict(dataset_dict["test"])
})

# Define the ner_tag_mapping
ner_tag_mapping = {
    'O': 0,
    'B-REDACTED': 1,
    'I-REDACTED': 2,
    'B-REDACTED': 3,
    'I-REDACTED': 4,
    'B-REDACTED': 5,
    'I-REDACTED': 6,
    'B-REDACTED': 7,
    'I-REDACTED': 8,
    'B-REDACTED': 9,
    'I-REDACTED': 10,
    'B-REDACTED': 11,
    'I-REDACTED': 12
}

# Define new features
new_features = Features({
    "id": datasets.Value("string"),
    "tokens": Sequence(datasets.Value("string")),
    "pos_tags": Sequence(datasets.Value("string")),
    "chunk_tags": Sequence(datasets.Value("string")),
    "ner_tags": Sequence(ClassLabel(names=list(ner_tag_mapping.keys())))
})

# Convert the dataset with new features
new_dataset = dataset.map(lambda x: x, features=new_features)

# Test to see if the changes reflect
print(new_dataset["train"].features["ner_tags"])


In [ ]:
new_dataset["train"].features["ner_tags"]

In [ ]:
new_dataset

In [ ]:
new_dataset.shape

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

Problem of consecutive subwords.
Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer.
This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain.

This is happening, first because some special tokens might be added (we can a [CLS] and a [SEP] above) and then because of those possible splits of words in multiple tokens:
Strategy to handle above - Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

Setting –100 as the label for these special tokens and the subwords we wish to mask during training:

Why did we choose –100 as the ID to mask subword representations? The reason is that in PyTorch the cross-entropy loss class torch.nn.CrossEntropyLoss has an attribute called ignore_index whose value is –100. This index is ignored during training,

Also we can use it to ignore the tokens associated with consecutive subwords.

In [ ]:
example_text = new_dataset['train'][0]

tokenized_input = tokenizer(example_text["tokens"], is_split_into_words=True)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

word_ids = tokenized_input.word_ids()

print(word_ids)
tokenized_input


Problem of Sub-Token - The input ids returned by the tokenizer are longer than the lists of labels our dataset contain.


In [ ]:
len(example_text['ner_tags']), len(tokenized_input["input_ids"])

The below function tokenize_and_align_labels does 2 jobs

    set –100 as the label for these special tokens and the subwords we wish to mask during training
    mask the subword representations after the first subword

Then we align the labels with the token ids using the strategy we picked:

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    """
    Function to tokenize and align labels with respect to the tokens. This function is specifically designed for
    Named Entity Recognition (NER) tasks where alignment of the labels is necessary after tokenization.

    Parameters:
    examples (dict): A dictionary containing the tokens and the corresponding NER tags.
                     - "tokens": list of words in a sentence.
                     - "ner_tags": list of corresponding entity tags for each word.

    label_all_tokens (bool): A flag to indicate whether all tokens should have labels.
                             If False, only the first token of a word will have a label,
                             the other tokens (subwords) corresponding to the same word will be assigned -100.

    Returns:
    tokenized_inputs (dict): A dictionary containing the tokenized inputs and the corresponding labels aligned with the tokens.
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
q = tokenize_and_align_labels(new_dataset['train'][4:5])
print(q)

So before applying the tokenize_and_align_labels() the tokenized_input has 3 keys

    input_ids
    token_type_ids
    attention_mask

But after applying tokenize_and_align_labels() we have an extra key - 'labels'~

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]),q["labels"][0]):
    print(f"{token:_<40} {label}")

In [ ]:
tokenized_datasets = new_dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=13)

In [ ]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=3,
weight_decay=0.01,
)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = datasets.load_metric("seqeval")

In [ ]:
example = new_dataset['train'][0]

In [ ]:
label_list = new_dataset["train"].features["ner_tags"].feature.names
label_list

In [ ]:
labels = [label_list[i] for i in example["ner_tags"]]

metric.compute(predictions=[labels], references=[labels])


# seqeval - The way the package works by accepting list of lists

The seqeval package expects the predictions and labels as lists of lists, with each list corresponding to a single example in our validation or test sets. To integrate these metrics during training, we need a function that can take the outputs of the model and convert them into the lists that seqeval expects.

The following does the trick by ensuring we ignore the label IDs associated with subsequent subwords:
# Compute Metrics

This compute_metrics() function first takes the argmax of the logits to convert them to predictions (as usual, the logits and the probabilities are in the same order, so we don’t need to apply the softmax). Then we have to convert both labels and predictions from integers to strings. We remove all the values where the label is -100, then pass the results to the metric.compute() method:
*italicised text*

In [ ]:
def compute_metrics(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
   "precision": results["overall_precision"],
   "recall": results["overall_recall"],
   "f1": results["overall_f1"],
  "accuracy": results["overall_accuracy"],
  }


# predictions will print a long 2d tensor like below

[['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-LOC', 'O', 'O', 'O', 'O', 'O'], ['B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'B-ORG', 'O', ['O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'B-ORG', 'O', 'B-ORG', 'I-ORG', 'O', 'B-ORG', 'B-ORG', 'B-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],

---

---

, ['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]

*italicised text*

In [ ]:
trainer = Trainer(
    model,
    args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

model.save_pretrained('REDACTED')

tokenizer.save_pretrained("/REDACTED")


In [ ]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [ ]:
import json

config = json.load(open("REDACTED.json"))

config["id2label"] = id2label
config["label2id"] = label2id

json.dump(config, open("REDACTED.json","w"))



In [ ]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("REDACTED")

In [ ]:
from transformers import pipeline
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)
ner_results = nlp(example)
print(ner_results)
